In [1]:
import cv2
import numpy as np


def text_to_binary(text):
    binary_text = ''.join(format(ord(char), '08b') for char in text)
    return binary_text


def encode_text(img, text):
    binary_text = text_to_binary(text)
    # Adding a delimiter to mark the end of the text
    binary_text += '1111111111111110'

    data_index = 0

    img_data = img.flatten()

    for pixel_value in img_data:
        # Encode the text in the least significant bit of each color channel
        img_data[data_index] = int(format(pixel_value, '08b')[
                                   :-1] + binary_text[data_index % len(binary_text)], 2)
        data_index += 1

        # Break if all data is encoded
        if data_index == len(binary_text):
            break

    return img_data.reshape(img.shape)


def normalize_color_spacing(img):
    # Convert image to LAB color space and normalize the values
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    lab_img[:, :, 0] = cv2.normalize(
        lab_img[:, :, 0], None, 0, 255, cv2.NORM_MINMAX)
    normalized_img = cv2.cvtColor(lab_img, cv2.COLOR_LAB2BGR)

    return normalized_img


# Example usage
image_path = 'one.png'
original_image = cv2.imread(image_path)

# Normalize color spacing
normalized_image = normalize_color_spacing(original_image)

# Encode text into the normalized image
secret_message = "Hello, this is a secret message!"
encoded_image = encode_text(normalized_image, secret_message)

# Save the encoded image
cv2.imwrite('encoded_image.png', encoded_image)

True

In [3]:
def decode_text(img):
    binary_text = ''
    img_data = img.flatten()

    for pixel_value in img_data:
        binary_text += format(pixel_value, '08b')[-1]

        # Check for the delimiter indicating the end of the text
        if binary_text[-16:] == '1111111111111110':
            break

    # Convert binary text to ASCII
    text = ''.join(chr(int(binary_text[i:i+8], 2))
                   for i in range(0, len(binary_text)-16, 8))

    return text


# Load the encoded image (for demonstration purposes)
loaded_encoded_image = cv2.imread('upload/Screenshot (7).png')

# Decode the text from the loaded encoded image
decoded_message_loaded = decode_text(loaded_encoded_image)

print(f"Original Message: {secret_message}")
print(f"Decoded Message from Loaded Image: {decoded_message_loaded}")

Original Message: Hello, this is a secret message!
Decoded Message from Loaded Image:  ÿ
